In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.optimizers import Adam

In [2]:
tf.__version__
    

'2.10.1'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

Reference to Image data pre-processing with generators  
https://www.geeksforgeeks.org/cnn-image-data-pre-processing-with-generators/  
https://msalamiitd.medium.com/how-to-pass-image-datasets-to-cnn-models-using-image-data-generations-b2d9497c7a35

In [2]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True
)
training_set = train_datagen.flow_from_directory(
    'seg_train/seg_train',
    target_size= (155,155),
    batch_size= 32,
    class_mode= 'categorical'
    )

Found 14034 images belonging to 6 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    'seg_test/seg_test',
    target_size= (155,155),
    batch_size= 32,
    class_mode= 'categorical'
    )


Found 3000 images belonging to 6 classes.


## Building CNN

In [6]:
input_shape = (155, 155, 3)
model = keras.Sequential(
    [
        
        layers.Conv2D(32, 3, input_shape = input_shape, activation='relu', padding="same", strides=1),
        layers.Conv2D(32, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Conv2D(128, 3, activation='relu', padding="same", strides=1),
        layers.Conv2D(128, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Reshape([-1]),
        layers.Dense(16, activation="relu"),
        layers.Dense(6, activation="softmax"),
    ]
)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 155, 155, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 155, 155, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 77, 77, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 77, 77, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 77, 77, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 64)       0         
 2D)                                                    

In [8]:
optimizer = Adam(learning_rate=0.001)

In [9]:
model.compile(optimizer=optimizer, 
            loss='categorical_crossentropy', 
            metrics=['accuracy'])

In [10]:
history = model.fit(x=training_set, validation_data=test_set, epochs=10, batch_size=32) 

Epoch 1/10
439/439 [==============================] - 95s 203ms/step - loss: 1.1476 - accuracy: 0.5363 - val_loss: 1.1154 - val_accuracy: 0.5923
Epoch 2/10
409/439 [==========================>...] - ETA: 4s - loss: 0.8317 - accuracy: 0.6846

## Building CNN (2)

In [6]:
input_shape = (155, 155, 3)

cnn = Sequential([
    Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
    MaxPool2D(pool_size=2, strides=2),
    Conv2D(filters=32, kernel_size=3, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Flatten(),
    Dense(units=10, activation='relu'),
    Dense(units=6, activation='softmax')
])


In [7]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 153, 153, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 76, 76, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 74, 74, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 43808)             0         
                                                                 
 dense (Dense)               (None, 10)                4

In [8]:
cnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn.fit(x=training_set, validation_data=test_set, epochs=50, batch_size=32)

## Save the model

In [ ]:
# model.save('model.h5')

## Load the model

In [5]:
import pickle
from tensorflow.keras.models import load_model

In [6]:
model = load_model('model.h5')

## Prediction 

In [4]:
from keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import random

class_indices = training_set.class_indices
class_names = list(class_indices.keys())

files = os.listdir('seg_pred')

def display_random_image_and_predict():
    
    random_file = random.choice(files)
    test_random_image_path = os.path.join('seg_pred', random_file)

    test_image = load_img(test_random_image_path, target_size = (155, 155))
    
    plt.imshow(test_image)
    plt.title("Loaded Image")
    plt.axis('off')
    plt.show()
    
    test_image_array = img_to_array(test_image)
    test_image_array = np.expand_dims(test_image_array, axis=0)
    
    result = cnn.predict(test_image_array)

    predicted_class_index = np.argmax(result, axis=1)[0]
    predicted_class_name = class_names[predicted_class_index]
    print(f"Predicted class: {predicted_class_name} (index: {predicted_class_index})")

In [ ]:
display_random_image_and_predict()